# **Extract, Transform, Load (ETL) Process**

## Objectives

* The objective of this notebook is to perform an ETL process of the data I have gathered from Kaggle.
* I have fetched the data which you can find in the datasets/raw folder.
* There are 4 related csv files that contain foreign keys that relate to eachother such as player_id, match_id and tournament_id. I will use these keys to clean and merge the data.
## Inputs

* My inputs are 4 csv files:
    * matches.csv
    * players.csv
    * scores.csv
    * tournaments.csv

## Outputs

* I am going to output one main table. From this table I will filter information into smaller csv files where I can work on them more easily to create visualisations.

## Additional Comments

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir


'/Users/stephenbeese/GitHub/Snooker-Data-Analysis/Snooker-Data-Analysis/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/Users/stephenbeese/GitHub/Snooker-Data-Analysis/Snooker-Data-Analysis'

# Imports

To clean and merge this data I will first need to import some libraries

In [5]:
import numpy as np
import pandas as pd

# Section 1 - Load the Data into DataFrames

Step 1 - Load data from datasets/raw folder into data frames

In [12]:
raw_data_dir = os.path.join(current_dir, 'datasets/raw')

df_matches = pd.read_csv(os.path.join(raw_data_dir, 'matches.csv'))
df_players = pd.read_csv(os.path.join(raw_data_dir, 'players.csv'))
df_scores = pd.read_csv(os.path.join(raw_data_dir, 'scores.csv'))
df_tournaments = pd.read_csv(os.path.join(raw_data_dir, 'tournaments.csv'))


In [ ]:
# Display the head of df_matches
df_matches.head()

,id,season,year,name,full_name,url,status,category,prize,country,city
0,753,1982-1983,1982,UK Championship,1982 UK Championship,https://cuetracker.net/tournaments/uk-champion...,Professional,Non-ranking,47000.0,England,Preston
1,1140,1982-1983,1982,World Amateur Championship - Men,1982 World Amateur Championship - Men,https://cuetracker.net/tournaments/world-amate...,Amateur,World Event,0.0,Canada,Calgary
2,762,1982-1983,1982,Professional Players Tournament,1982 Professional Players Tournament,https://cuetracker.net/tournaments/professiona...,Professional,Ranking,31500.0,England,Birmingham
3,2586,1982-1983,1982,Pontins Autumn Open,1982 Pontins Autumn Open,https://cuetracker.net/tournaments/pontins-aut...,Pro-am,Event,0.0,Wales,Prestatyn
4,754,1982-1983,1982,International Open,1982 International Open,https://cuetracker.net/tournaments/internation...,Professional,Ranking,73500.0,England,Derby
...,...,...,...,...,...,...,...,...,...,...,...
2715,2884,2018-2019,2019,Guernsey Amateur Championship,2019 Guernsey Amateur Championship,https://cuetracker.net/tournaments/guernsey-am...,Amateur,National Championship,0.0,Guernsey,Various
2716,3321,2019-2020,2020,English Amateur Championship,2020 English Amateur Championship,https://cuetracker.net/tournaments/english-ama...,Amateur,National Championship,0.0,England,Cheltenham
2717,3362,NaN,2020,3 Kings Open,2020 3 Kings Open,https://cuetracker.net/tournaments/3-kings-ope...,Pro-am,Event,0.0,Austria,Rankweil
2718,3357,2019-2020,2020,Singapore Amateur Championship,2020 Singapore Amateur Championship,https://cuetracker.net/tournaments/singapore-a...,Amateur,National Championship,0.0,Singapore,Singapore


---

# Section 2

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
